In [3]:
import numpy as np
from src.datasets import NoisySpectralSpeechDataset
from src.utils import checkpoint, spectral, notebook

In [4]:
dataset = NoisySpectralSpeechDataset(train=False, quiet=True)

In [5]:
CHECKPOINT = 'spectral-u-net-spectal-first-1575698669.full.ckpt'
# CHECKPOINT = 'spectral-u-net-spectral-max-pool-1576312399.full.ckpt'
CHECKPOINT = 'spectral-u-net-spectral-mask-1576312649.full.ckpt'

In [6]:
net = checkpoint.load(CHECKPOINT, use_cuda=False)
net = net.eval()

In [7]:
def reshape_spectrum(arr):
    assert arr.shape == (2, 256, 128)
    arr = np.concatenate([arr, np.zeros([2, 1, 128])], axis=1)
    assert arr.shape == (2, 257, 128)
    arr = np.concatenate([arr, np.zeros([2, 257, 1])], axis=2)
    assert arr.shape == (2, 257, 129)
    return arr

In [8]:
IDX = 126

noisy_spectral, clean_spectral = dataset[IDX]
noisy_arr = spectral.spec_to_audio(reshape_spectrum(noisy_spectral.numpy()))
clean_arr = spectral.spec_to_audio(reshape_spectrum(clean_spectral.numpy()))

In [9]:
noisy_spectral_batch = noisy_spectral.view(1, 2, 256, 128)
pred_clean_spectral_batch = net(noisy_spectral_batch)
pred_clean_spectral = pred_clean_spectral_batch.view(2, 256, 128)

RuntimeError: Given groups=1, weight of size 24 2 15 15, expected input[1, 1, 256, 128] to have 2 channels, but got 1 channels instead

In [ ]:
pred_clean_arr = spectral.spec_to_audio(reshape_spectrum(pred_clean_spectral.detach().numpy()))

In [ ]:
notebook.visualize_audio(pred_clean_arr, 'Pred clean signal')

In [ ]:
notebook.visualize_audio(dataset.noisy_data[IDX], 'Noisy signal')

In [ ]:
notebook.visualize_audio(dataset.clean_data[IDX], 'Clean signal')

In [ ]:
notebook.visualize_audio(clean_arr, 'Clean signal reconstructed')